# Imports

In [1]:
import tensorflow_decision_forests as tfdf
import numpy as np
import pandas as pd
import tensorflow as tf

2023-09-23 19:32:12.562858: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 19:32:12.608292: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 19:32:14.011013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Treinamento

## Carregando e preparando dataset

In [3]:
dataframe = pd.read_csv("dummy-data.csv")
dataframe.head(3)

,idade,sexo,profissao,renda,seguro
0,51,Masculino,Psicólogo Clínico,5000,Vida Familia
1,29,Feminino,Gerente de Projetos,8000,Fatura Protegida
2,23,Feminino,Cientista de Dados,8500,Carro


In [4]:
target = "seguro"
classes = dataframe[target].unique().tolist()
print(f"Classes: {classes}")
dataframe[target] = dataframe[target].map(classes.index)

Classes: ['Vida Familia', 'Fatura Protegida', 'Carro', 'Moto', 'Proteção Pessoal', 'Celular']


In [5]:
def split_dataframe(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataframe(dataframe)
print("{} linhas para treinamento, {} linhas para teste.".format(
    len(train_ds_pd), len(test_ds_pd)))

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=target)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=target)

14081 linhas para treinamento, 5919 linhas para teste.


2023-09-23 19:32:53.250143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-23 19:32:53.251798: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Criando e treinando Modelo

In [7]:
modelo = tfdf.keras.RandomForestModel(verbose=2, num_trees=20)
modelo.fit(train_ds)

Use 8 thread(s) for training
Use /tmp/tmpeburcdgu as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'idade': <tf.Tensor 'data:0' shape=(None,) dtype=int64>, 'sexo': <tf.Tensor 'data_1:0' shape=(None,) dtype=string>, 'profissao': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'renda': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>}
Label: Tensor("data_4:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'idade': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'sexo': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data_1:0' shape=(None,) dtype=string>), 'profissao': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data_2:0' shape=(None,) dtype=string>), 'renda': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_1:0' shape=(None,) dtype=float32>)}
Training dataset read in 0:00:03.892268. 

[INFO 23-09-23 19:33:07.7095 -04 kernel.cc:773] Start Yggdrasil model training
[INFO 23-09-23 19:33:07.7095 -04 kernel.cc:774] Collect training examples
[INFO 23-09-23 19:33:07.7095 -04 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-09-23 19:33:07.7099 -04 kernel.cc:393] Number of batches: 15
[INFO 23-09-23 19:33:07.7099 -04 kernel.cc:394] Number of examples: 14081
[INFO 23-09-23 19:33:07.7124 -04 kernel.cc:794] Training dataset:
Number of records: 14081
Number of columns: 5

Number of columns by type:
	CATEGORICAL: 3 (60%)
	NUMERICAL: 2 (40%)

Columns:

CATEGORICAL: 3 (60%)
	0: "__LABEL" CATEGORICAL integerized vocab-size:7 no-ood-item
	2: "p

Model trained in 0:00:00.428277
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


## Avaliando modelo

In [8]:
modelo.compile(metrics=["accuracy"])
evaluation = modelo.evaluate(test_ds, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

6/6 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.3923
loss: 0.0000
accuracy: 0.3923


In [9]:
tfdf.model_plotter.plot_model_in_colab(modelo, tree_idx=0, max_depth=3)

In [10]:
predict = modelo.predict({
    "idade": np.array([36]),
    "sexo": np.array(["Masculino"]),
    "profissao": np.array(["Médica"]),
    "renda": np.array([10000]),
})[:1]
list(zip(predict[0].tolist(), (classes)))

1/1 [==============================] - 0s 44ms/step


[(0.25, 'Vida Familia'),
 (0.0, 'Fatura Protegida'),
 (0.25, 'Carro'),
 (0.0, 'Moto'),
 (0.5000000596046448, 'Proteção Pessoal'),
 (0.0, 'Celular')]

## Salvando modelo e testando carregamento

In [11]:
modelo.save("modelo")

INFO:tensorflow:Assets written to: modelo/assets


INFO:tensorflow:Assets written to: modelo/assets


In [12]:
modelo_teste = tf.keras.models.load_model("modelo")
predict = modelo_teste.predict({
    "idade": np.array([36]),
    "sexo": np.array(["Masculino"]),
    "profissao": np.array(["Médica"]),
    "renda": np.array([10000]),
})[:1]
list(zip(predict[0].tolist(), (classes)))

[INFO 23-09-23 19:33:39.4025 -04 kernel.cc:1243] Loading model from path modelo/assets/ with prefix 2d10c06ba130401d
[INFO 23-09-23 19:33:39.5781 -04 decision_forest.cc:660] Model loaded with 20 root(s), 62212 node(s), and 4 input feature(s).
[INFO 23-09-23 19:33:39.5782 -04 abstract_model.cc:1311] Engine "RandomForestGeneric" built
[INFO 23-09-23 19:33:39.5782 -04 kernel.cc:1075] Use fast generic engine


1/1 [==============================] - 0s 71ms/step


[(0.25, 'Vida Familia'),
 (0.0, 'Fatura Protegida'),
 (0.25, 'Carro'),
 (0.0, 'Moto'),
 (0.5000000596046448, 'Proteção Pessoal'),
 (0.0, 'Celular')]